# **Author: Vijay Varma - AI20BTECH11012**

# **Kaggle Display Name : AI20BTECH11012**

# **Model 2 using Random Forest**

# **Downloading Data from Kaggle**

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c new-york-city-taxi-fare-prediction

  0% 0.00/960k [00:00<?, ?B/s]
100% 960k/960k [00:00<00:00, 65.6MB/s]
  0% 0.00/486 [00:00<?, ?B/s]
100% 486/486 [00:00<00:00, 480kB/s]
100% 1.56G/1.56G [00:39<00:00, 37.6MB/s]
100% 1.56G/1.56G [00:39<00:00, 42.1MB/s]
  0% 0.00/335k [00:00<?, ?B/s]
100% 335k/335k [00:00<00:00, 103MB/s]


In [6]:
! unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [7]:
#### 
# Importing Required Packages
import pandas as pd
import numpy as np
import datetime as dt

In [8]:
####
# Loading 10000000 rows of tran.csv into df_train

df_train = pd.read_csv("train.csv",nrows=1000000)

In [9]:
####
# Printing df_train

df_train

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
999995,2014-09-13 21:44:38.0000001,7.0,2014-09-13 21:44:38 UTC,-73.976676,40.785630,-73.959196,40.775114,1
999996,2010-09-20 14:50:37.0000002,7.3,2010-09-20 14:50:37 UTC,-73.992103,40.671385,-73.992103,40.671385,2
999997,2013-04-26 14:03:00.000000118,10.5,2013-04-26 14:03:00 UTC,-73.978423,40.751135,-73.993397,40.762197,5
999998,2011-07-08 00:29:00.00000099,6.9,2011-07-08 00:29:00 UTC,-73.980317,40.759482,-73.976832,40.743122,1


In [10]:
####
# Loading test.csv into df_test

df_test = pd.read_csv("test.csv")

In [11]:
####
# Printing df_test

df_test

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1
...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6


In [12]:
#### 
# Copying train_df, test_df into another dataframes

train_df = df_train.copy()
test_df = df_test.copy()

In [13]:
####
# Let's get an understanding of our data
# Let's print a snapshot of our train data frame 
# train data frame

train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
999995,2014-09-13 21:44:38.0000001,7.0,2014-09-13 21:44:38 UTC,-73.976676,40.785630,-73.959196,40.775114,1
999996,2010-09-20 14:50:37.0000002,7.3,2010-09-20 14:50:37 UTC,-73.992103,40.671385,-73.992103,40.671385,2
999997,2013-04-26 14:03:00.000000118,10.5,2013-04-26 14:03:00 UTC,-73.978423,40.751135,-73.993397,40.762197,5
999998,2011-07-08 00:29:00.00000099,6.9,2011-07-08 00:29:00 UTC,-73.980317,40.759482,-73.976832,40.743122,1


In [14]:
####
# Printing types of columns in train data frame

print(train_df.dtypes)

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object


# **We have to add 'year', 'month', 'day', 'hour' as features from 'pickup_datetime' column into our train and test data frame.**

In [15]:
####
# This function adds 'year', 'month', 'day', 'hour' as features from 'pickup_datetime' column into the data frame

def add_date_time(df):
    # splits 'pickup_datetime' into 'year', 'month', 'day'
    df[['year', 'month', 'day']] = df.pickup_datetime.str.split(' ', expand=True).iloc[:,0].str.split('-', expand=True).astype('int64')
    # adds hour into the data frame
    df['hour'] = df.pickup_datetime.str.split(' ', expand=True).iloc[:,1].str.split(':', expand=True).iloc[:,0].astype('int64')

    return df

In [16]:
####
# Adds 'year', 'month', 'day', 'hour' as features from 'pickup_datetime' column into the train_df

train_df = add_date_time(train_df)

In [17]:
####
# Printing modified train_df

train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2014-09-13 21:44:38.0000001,7.0,2014-09-13 21:44:38 UTC,-73.976676,40.785630,-73.959196,40.775114,1,2014,9,13,21
999996,2010-09-20 14:50:37.0000002,7.3,2010-09-20 14:50:37 UTC,-73.992103,40.671385,-73.992103,40.671385,2,2010,9,20,14
999997,2013-04-26 14:03:00.000000118,10.5,2013-04-26 14:03:00 UTC,-73.978423,40.751135,-73.993397,40.762197,5,2013,4,26,14
999998,2011-07-08 00:29:00.00000099,6.9,2011-07-08 00:29:00 UTC,-73.980317,40.759482,-73.976832,40.743122,1,2011,7,8,0


In [18]:
####
# Let's get an understanding of our data
# Let's print a snapshot of our test data frame 
# test data frame

test_df

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1
...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6


In [19]:
####
# Printing types of columns in test data frame

print(test_df.dtypes)

key                   object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object


In [20]:
####
# Adds 'year', 'month', 'day', 'hour' as features from 'pickup_datetime' column into the train_df

test_df = add_date_time(test_df)

In [21]:
####
# Printing modified test_df

test_df

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21
...,...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6,2015,5,10,12
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6,2015,1,12,17
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6,2015,4,19,20
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6,2015,1,31,1


In [22]:
####
# Dropping Columns "pickup_datetime","key" from train_df as they are not used as features in our model

train_df.drop(columns=["pickup_datetime","key"], axis=1, inplace=True)

In [23]:
####
# Dropping Column "pickup_datetime" from test_df as it is not used as feature in our model

test_df.drop(columns=["pickup_datetime"], axis=1, inplace=True)

In [24]:
####
# Printing modified train_df

train_df

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7
...,...,...,...,...,...,...,...,...,...,...
999995,7.0,-73.976676,40.785630,-73.959196,40.775114,1,2014,9,13,21
999996,7.3,-73.992103,40.671385,-73.992103,40.671385,2,2010,9,20,14
999997,10.5,-73.978423,40.751135,-73.993397,40.762197,5,2013,4,26,14
999998,6.9,-73.980317,40.759482,-73.976832,40.743122,1,2011,7,8,0


In [25]:
####
# Printing modified test_df

test_df

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21
...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,-73.968124,40.796997,-73.955643,40.780388,6,2015,5,10,12
9910,2015-01-12 17:05:51.0000001,-73.945511,40.803600,-73.960213,40.776371,6,2015,1,12,17
9911,2015-04-19 20:44:15.0000001,-73.991600,40.726608,-73.789742,40.647011,6,2015,4,19,20
9912,2015-01-31 01:05:19.0000005,-73.985573,40.735432,-73.939178,40.801731,6,2015,1,31,1


# **Check if there are any Null Values in any column of our data**

In [26]:
####
# Checking if there are any Null Values in any column of our train_df

train_df.isnull().sum()

fare_amount           0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    10
dropoff_latitude     10
passenger_count       0
year                  0
month                 0
day                   0
hour                  0
dtype: int64

In [27]:
####
# Checking if there are any Null Values in any column of our test_df

test_df.isnull().sum()

key                  0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
year                 0
month                0
day                  0
hour                 0
dtype: int64

In [28]:
####
# Dropping Null Values in the train data

train_df.dropna(inplace=True)

In [29]:
####
# Printing modified train_df

train_df

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7
...,...,...,...,...,...,...,...,...,...,...
999995,7.0,-73.976676,40.785630,-73.959196,40.775114,1,2014,9,13,21
999996,7.3,-73.992103,40.671385,-73.992103,40.671385,2,2010,9,20,14
999997,10.5,-73.978423,40.751135,-73.993397,40.762197,5,2013,4,26,14
999998,6.9,-73.980317,40.759482,-73.976832,40.743122,1,2011,7,8,0


In [30]:
####
# Lets look at the statistics of our train data

train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
count,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000
mean,11.347953,-72.526699,39.929040,-72.527860,39.919954,1.684941,2011.741103,6.267841,15.697996,13.509463
std,9.821790,12.057778,7.626087,11.324494,8.201418,1.323907,1.860761,3.436230,8.682472,6.513855
min,-44.900000,-3377.680935,-3116.285383,-3383.296608,-3114.338567,0.000000,2009.000000,1.000000,1.000000,0.000000
25%,6.000000,-73.992060,40.734965,-73.991385,40.734046,1.000000,2010.000000,3.000000,8.000000,9.000000
50%,8.500000,-73.981792,40.752695,-73.980135,40.753166,1.000000,2012.000000,6.000000,16.000000,14.000000
75%,12.500000,-73.967094,40.767154,-73.963654,40.768129,2.000000,2013.000000,9.000000,23.000000,19.000000
max,500.000000,2522.271325,2621.628430,45.581619,1651.553433,208.000000,2015.000000,12.000000,31.000000,23.000000


In [31]:
####
# Printing test_df

test_df

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21
...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,-73.968124,40.796997,-73.955643,40.780388,6,2015,5,10,12
9910,2015-01-12 17:05:51.0000001,-73.945511,40.803600,-73.960213,40.776371,6,2015,1,12,17
9911,2015-04-19 20:44:15.0000001,-73.991600,40.726608,-73.789742,40.647011,6,2015,4,19,20
9912,2015-01-31 01:05:19.0000005,-73.985573,40.735432,-73.939178,40.801731,6,2015,1,31,1


In [32]:
####
# Lets look at the statistics of our test data

test_df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273,2011.815816,6.857979,16.194170,13.467420
std,0.042774,0.033541,0.039072,0.035435,1.278747,1.803347,3.353272,8.838482,6.868584
min,-74.252193,40.573143,-74.263242,40.568973,1.000000,2009.000000,1.000000,1.000000,0.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000,2010.000000,4.000000,9.000000,8.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000,2012.000000,7.000000,16.000000,15.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000,2014.000000,10.000000,25.000000,19.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000,2015.000000,12.000000,31.000000,23.000000


# **There are many Outliers in our train data.**

# **We have to remove them so that our model performs better.**

# **We will remove the Outliers Column by Column.**

# **1. Passenger Count**

# **We drop the rows in train data where passenger_count is more than 6 (practically not possible).**

In [33]:
####
# Dropping the Rows where passenger_count > 6 in train_df

train_df = train_df.drop(train_df[train_df['passenger_count']>6].index, axis=0)

In [34]:
####
# Printing modified train_df

train_df

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7
...,...,...,...,...,...,...,...,...,...,...
999995,7.0,-73.976676,40.785630,-73.959196,40.775114,1,2014,9,13,21
999996,7.3,-73.992103,40.671385,-73.992103,40.671385,2,2010,9,20,14
999997,10.5,-73.978423,40.751135,-73.993397,40.762197,5,2013,4,26,14
999998,6.9,-73.980317,40.759482,-73.976832,40.743122,1,2011,7,8,0


# **As the data is about taxi in New York City, we consider only the longitude and latitude in the range of New York City.**

# **The Latitude and Longitude of New York City is approximately taken from**

# **Latitude : (40.4,41.0)**

# **Longitude : (-74.2,-73.7)**

# **2. Pickup Longitude**

In [35]:
####
# Dropping 'pickup_longitude' other than in New York City

train_df = train_df.drop(train_df[train_df['pickup_longitude']<-74.2].index, axis=0)
train_df = train_df.drop(train_df[train_df['pickup_longitude']>-73.7].index, axis=0)

In [36]:
####
# Dropping 'dropoff_longitude' other than in New York City

train_df = train_df.drop(train_df[train_df['dropoff_longitude']<-74.2].index, axis=0)
train_df = train_df.drop(train_df[train_df['dropoff_longitude']>-73.7].index, axis=0)

In [37]:
####
# Dropping 'pickup_latitude' other than in New York City

train_df = train_df.drop(train_df[train_df['pickup_latitude']<40.4].index, axis=0)
train_df = train_df.drop(train_df[train_df['pickup_latitude']>41.0].index, axis=0)

In [38]:
####
# Dropping 'dropoff_latitude' other than in New York City

train_df = train_df.drop(train_df[train_df['dropoff_latitude']<40.4].index, axis=0)
train_df = train_df.drop(train_df[train_df['dropoff_latitude']>41.0].index, axis=0)

In [39]:
####
# Printing Modified train_df

train_df

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7
...,...,...,...,...,...,...,...,...,...,...
999995,7.0,-73.976676,40.785630,-73.959196,40.775114,1,2014,9,13,21
999996,7.3,-73.992103,40.671385,-73.992103,40.671385,2,2010,9,20,14
999997,10.5,-73.978423,40.751135,-73.993397,40.762197,5,2013,4,26,14
999998,6.9,-73.980317,40.759482,-73.976832,40.743122,1,2011,7,8,0


# **3. Fare Amount**

# **Generally Fare Amount lies in between 2 to 100.**

In [40]:
####
# Dropping 'fare_amount' other than 2 to 100

train_df = train_df.drop(train_df[train_df['fare_amount']<=2].index, axis=0)
train_df = train_df.drop(train_df[train_df['fare_amount']>100].index, axis=0)

In [41]:
####
# Printing Modified train_df

train_df

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7
...,...,...,...,...,...,...,...,...,...,...
999995,7.0,-73.976676,40.785630,-73.959196,40.775114,1,2014,9,13,21
999996,7.3,-73.992103,40.671385,-73.992103,40.671385,2,2010,9,20,14
999997,10.5,-73.978423,40.751135,-73.993397,40.762197,5,2013,4,26,14
999998,6.9,-73.980317,40.759482,-73.976832,40.743122,1,2011,7,8,0


# **Now, we have to split train_df into train_x (contains only features) and train_y (contains only labels).**

In [42]:
####
# Extracting 'fare_amount' Column into train_y list

train_y = train_df['fare_amount'].to_list()

# Dropping 'fare_amount' Column from train_x Data Frame

train_df.drop(columns=['fare_amount'],axis=1,inplace = True)

In [43]:
####
# Copying test_df into a new Data Frame

test_x = test_df.copy()

In [44]:
####
# Extracting 'key' Column into test_id list

test_key = test_df["key"].to_list()

# Dropping 'key' Column from test_x Data Frame

test_x.drop(columns=["key"],axis=1,inplace = True)

In [45]:
####
# Printing test_key

test_key

['2015-01-27 13:08:24.0000002',
 '2015-01-27 13:08:24.0000003',
 '2011-10-08 11:53:44.0000002',
 '2012-12-01 21:12:12.0000002',
 '2012-12-01 21:12:12.0000003',
 '2012-12-01 21:12:12.0000005',
 '2011-10-06 12:10:20.0000001',
 '2011-10-06 12:10:20.0000003',
 '2011-10-06 12:10:20.0000002',
 '2014-02-18 15:22:20.0000002',
 '2014-02-18 15:22:20.0000003',
 '2014-02-18 15:22:20.0000001',
 '2010-03-29 20:20:32.0000002',
 '2010-03-29 20:20:32.0000001',
 '2011-10-06 03:59:12.0000002',
 '2011-10-06 03:59:12.0000001',
 '2012-07-15 16:45:04.0000006',
 '2012-07-15 16:45:04.0000002',
 '2012-07-15 16:45:04.0000003',
 '2012-07-15 16:45:04.0000004',
 '2014-10-29 02:09:56.0000001',
 '2014-06-14 13:39:00.00000010',
 '2014-06-14 13:39:00.00000060',
 '2014-06-14 13:39:00.00000087',
 '2014-06-14 13:39:00.00000050',
 '2014-06-14 13:39:00.0000003',
 '2014-06-14 13:39:00.000000158',
 '2014-06-14 13:39:00.00000015',
 '2014-06-14 13:39:00.00000073',
 '2014-06-14 13:39:00.00000077',
 '2014-06-14 13:39:00.000000145

In [46]:
#### Converting train_x, train_y, test_x into numpy arrays

train_x_arr = train_df.to_numpy()

train_y_arr = np.array(train_y)

test_x_arr = test_x.to_numpy()

# **Implementing the Model**

# **Random Forest Regressor**

In [47]:
####
# Importing Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor()

# Training the Regressor

clf.fit(train_x_arr,train_y_arr)

RandomForestRegressor()

In [48]:
####
# Predicting test_x_arr using the Regressor

predictions = clf.predict(test_x_arr)

print(predictions)

[ 9.3    11.11    4.308  ... 54.3391 15.93    7.255 ]


In [49]:
####
# Attaching test_key to predictions and adding them in submission data frame

submission = pd.DataFrame()
submission['key'] = test_key
submission['fare_amount'] = list(predictions)

In [50]:
####
# Prnting the Submission

submission

,key,fare_amount
0,2015-01-27 13:08:24.0000002,9.3000
1,2015-01-27 13:08:24.0000003,11.1100
2,2011-10-08 11:53:44.0000002,4.3080
3,2012-12-01 21:12:12.0000002,8.8160
4,2012-12-01 21:12:12.0000003,15.7390
...,...,...
9909,2015-05-10 12:37:51.0000002,9.9900
9910,2015-01-12 17:05:51.0000001,10.9250
9911,2015-04-19 20:44:15.0000001,54.3391
9912,2015-01-31 01:05:19.0000005,15.9300


In [51]:
####
# Saving submission into a csv file

submission.to_csv('model_submission_2.csv',index=False)

In [52]:
####
# Submitting submission to kaggle

! kaggle competitions submit -c new-york-city-taxi-fare-prediction -f model_submission_2.csv -m "Random Forest Regressor "

100% 402k/402k [00:02<00:00, 203kB/s]
Successfully submitted to New York City Taxi Fare Prediction